# 신경망 모델 구성
- 신경망은 데이터에 대한 연산을 수행하는 layer와 module로 구성되어 있습니다.
- torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성요소를 제공
- Pytorch의 모든 모듈은 nn.Module의 subclass 입니다. 
- 신경망은 다른 모듈 및 계층으로 구성된 모듈
- 이렇게 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

In [2]:
import os 
import torch
from torch import nn
from torch.utils.data import DataLoader 
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기 
- GPU에서 모델을 학습시키기 위해서 사용, torch.cuda를 사용할 수 있는지 확인하고 그렇지 않다면 CPU 사용

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


# 클래스 정의하기 
- 신경망 모델을 nn.Module의 subclass로 정의하고 __init__에서 신경망 layer들을 초기화합니다. 
- nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현합니다.

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [5]:
# 정의한 클래스의 인스턴스를 생성하고 이를 device로 이동한뒤 구조를 출력
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [7]:
# 모델을 사용하기 위해 입력 데이터를 전달, 
# 일부 백그라운드 연산(https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866)과 함께 모델의 
# forword를 실행 

# 모델에 입력을 호출하면 각 class에 대한 raw 에측 값이 있는 10차원 텐서가 반환
# 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻는다.

x = torch.rand(1, 28, 28, device = device)
logits = model(x)
pred_probab = nn.Softmax(dim = 1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class : {y_pred}")

Predicted class : tensor([8])


## 모델 계층(layer)
- FashionMNIST 모델에서 28x28 크기의 이미지 3개를 미니배치로 가져와 신경망을 통과할 때 어떤 일이 발생하는지 알아보겠음 

In [8]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
# nn.Flatten
# nn.Flatten은 28x28의 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환 (dim = 0의 미니배치 차원은 유지)
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
# nn.Linear
# 선형 계층은 저장된 weight와 bias를 사용하여 입력에 linear transformation을 적용하는 모듈 

layer1 = nn.Linear(in_features = 28*28, out_features = 28)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 28])


# nn.Sequential 
- nn.Sequential은 순서를 갖는 모듈의 컨테이너입니다
- 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달 됩니다.
- sequential container를 사용하여 아래의 seq_modules와 같은 신경망을 빠르게 만들 수 있습니다.

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(28, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

# nn.Softmax 
- 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 raw value값인) logits을 반환 
- logits는 모델의 각 class에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 scale 됩니다. 
- dim 매개변수는 값의 합이 1이 되는 차원을 나타냄

In [14]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

# 모델 매개변수
- 신경망 내부 계층들은 parameterize 됩니다. 즉, 학습 중에 최적화되는 weight 및 bias와 연관지어 집니다.
- nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 track 되며, 
모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근 할 수 있음

In [ ]:
# 각 매개변수들을 (순회)iterate 및 크기와 값을 출력
print("Model structures : ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer : {name} | Size : {param.size()} | Values : {param[:2]} \n")